# Diagrams in Python

This notebook is intended to show how you can easily create and manipulate diagrams directly in Python code. It uses the [Diagrams](https://diagrams.mingrammer.com/) Python package.

## Install the prerequisites

In [ ]:
import sys
!conda install --yes --prefix {sys.prefix} diagrams

# Import all of the node types from Diagrams

In [ ]:
from diagrams import Cluster, Diagram, Edge
from diagrams.custom import Custom
from diagrams.aws.compute import EC2
from diagrams.aws.compute import Lambda
from diagrams.aws.integration import SQS
from diagrams.aws.storage import S3
from diagrams.aws.analytics import KinesisDataFirehose
from diagrams.aws.analytics import Kinesis
from diagrams.aws.compute import ECS

from diagrams.aws.management import Cloudwatch

# Import `display` so we can show the image when we're done

In [ ]:
from IPython import display

# Define our diagram and display it

In [ ]:
with Diagram("test", show=False):

    scrubbed_s3 = S3("scrubbed events\nprod")
    historical_events_backfill = SQS("historical event\nqueue backfill")
    replay_lambda = Lambda("historical\nevent replay\nlambda")
    timer = Cloudwatch("timer trigger")

    with Cluster("downstream consumers"):
        downstream = [ECS(), ECS(), EC2()]

    Custom("manual trigger", "manual.png") \
            >> Lambda("dw queue\nbackfill tasks") \
            >> [scrubbed_s3, historical_events_backfill] \
            >> Lambda("historical\nevent filter\ntransform\nlambda") \
            >> Kinesis("historical event\nreplay kinesis") \
            >> KinesisDataFirehose("historical event\nreplay firehose") \
            >> S3("historical event\nbackfill") \
            >> replay_lambda \
            >> Kinesis("enriched event") \
            >> downstream

    timer >> replay_lambda
    
display.Image("./test.png")